<p align="center"><img width="50%" src="https://aimodelsharecontent.s3.amazonaws.com/aimodshare_banner.jpg" /></p>


---

## Model Submission Guide: CapIQ-Rating Classification Competition
Let's share our models to a centralized leaderboard, so that we can collaborate and learn from the model experimentation process...

**Instructions:**
1.   Get data in and set up X_train / X_test / y_train
2.   Preprocess data using Sklearn Column Transformer/ Write and Save Preprocessor function
3. Fit model on preprocessed data and save preprocessor function and model 
4. Generate predictions from X_test data and submit model to competition
5. Repeat submission process to improve place on leaderboard



## 1. Get data in and set up X_train, X_test, y_train objects

In [ ]:
#install aimodelshare library
! pip install aimodelshare --upgrade

In [2]:
# Get competition data
from aimodelshare import download_data
download_data('public.ecr.aws/y2e2a1d6/capiq_rating_competition_data-repository:latest') 


Data downloaded successfully.


In [3]:
# Separate data into X_train, y_train, and X_test
import pandas as pd
y_train_labels = pd.read_csv("capiq_rating_competition_data/y_train.csv", squeeze=True)
y_train = pd.get_dummies(y_train_labels)

X_train = pd.read_csv("capiq_rating_competition_data/X_train.csv")
X_test=pd.read_csv("capiq_rating_competition_data/X_test.csv")

X_train.head()

,Name,Symbol,Exchange,Industry,MarketCap,EnterpriseValue,Revenue,GrossProfit,EBITDA,EBIT,...,CurrentAssets,ShortTermDebt,LTD_Cap_Leases,Leases_LongTerm,LongTermDebt,Liabilities,Liabilities_N_Equity,Debt_Current,Debt_NonCurrent,Debt_Net
0,3M Company (NYSE:MMM),NYSE:MMM,New York Stock Exchange (NYSE),Industrials,99704.48,109611.58,31841.90,15447.60,8734.60,7966.300,...,13272.00,353.7000,1117.30,253.1000,11248.20,31269.350,37798.60,1471.000,11501.30,9166.30
1,AAR Corp. (NYSE:AIR),NYSE:AIR,New York Stock Exchange (NYSE),Industrials,1131.61,1412.65,1816.20,272.93,129.26,103.655,...,1014.52,0.0600,37.37,13.0800,328.11,1303.615,1760.14,37.430,341.19,276.71
2,Abbott Laboratories (NYSE:ABT),NYSE:ABT,New York Stock Exchange (NYSE),Health Care,112198.80,121025.92,27682.23,15806.85,6757.35,5505.365,...,20018.30,1243.9000,608.10,261.3000,15047.10,46540.700,60436.20,1852.000,15308.40,8373.90
3,AbbVie Inc. (NYSE:ABBV),NYSE:ABBV,New York Stock Exchange (NYSE),Health Care,135936.95,166341.05,31122.80,23163.60,13948.20,12604.850,...,22157.20,678.8000,3891.50,186.2000,37593.70,70162.000,71921.50,4570.300,37779.90,30633.70
4,Adecoagro S.A. (NYSE:AGRO),NYSE:AGRO,New York Stock Exchange (NYSE),Consumer Staples,1021.64,1760.08,817.16,260.81,255.28,193.000,...,623.58,7.6627,178.79,53.7321,595.08,1513.135,1949.59,186.449,648.81,606.12


##2.   Preprocess data using Sklearn / Write and Save Preprocessor function


In [4]:
# Simple Preprocessor with sklearn 
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_features = ['MarketCap', 'EnterpriseValue', 'Revenue', 'GrossProfit', 'EBITDA', 
                    'EBIT', 'NetIncome', 'Cash', 'PPnE', 'Assets', 'Debt', 'Equity', 
                    'Receivables', 'Inventory', 'CurrentAssets', 'ShortTermDebt', 
                    'LTD_Cap_Leases', 'Leases_LongTerm', 'LongTermDebt', 'Liabilities', 
                    'Liabilities_N_Equity', 'Debt_Current', 'Debt_NonCurrent', 'Debt_Net']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')), 
    ('scaler', StandardScaler())])

categorical_features = ['Industry']
## Replacing missing values with Modal value and then one-hot encoding.
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Final preprocessor object set up with ColumnTransformer...

preprocess = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# fit preprocessor to your data
preprocess = preprocess.fit(X_train)

In [5]:
# Here is where we actually write the preprocessor function:
def preprocessor(data):
    preprocessed_data=preprocess.transform(data)
    return preprocessed_data

In [6]:
# check shape of X data after preprocessing it using our new function
preprocessor(X_train).shape

(406, 34)

##3. Fit model on preprocessed data and save preprocessor function and model 


In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(16, input_dim=34, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(32, activation='relu'))

model.add(Dense(16, activation='softmax')) 
                                            
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model.fit(preprocessor(X_train), y_train, 
               epochs = 50, validation_split=0.25) 

#### Save preprocessor function to local "preprocessor.zip" file

In [18]:
import aimodelshare as ai
ai.export_preprocessor(preprocessor,"") 

Your preprocessor is now saved to 'preprocessor.zip'


#### Save model to local ".onnx" file

In [19]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

## 4. Generate predictions from X_test data and submit model to competition


In [20]:
#Set credentials using modelshare.org username/password

from aimodelshare.aws import set_credentials

#This is the unique rest api that powers this Rating Classification Playground -- make sure to update the apiurl for new competition deployments
apiurl='https://3hf4nd1e6b.execute-api.us-east-1.amazonaws.com/prod/m'

set_credentials(apiurl=apiurl)

AI Modelshare Username:··········
AI Modelshare Password:··········
AI Model Share login credentials set successfully.


In [21]:
#Instantiate Competition
import aimodelshare as ai
mycompetition= ai.Competition(apiurl)

In [22]:
#Submit Model 1: 

#-- Generate predicted y values (Model 1)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 1 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 5

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1756


In [23]:
# Get leaderboard to explore current best model architectures

# Get raw data in pandas data frame
data = mycompetition.get_leaderboard()

# Stylize leaderboard data
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dense_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,32.00%,23.20%,28.63%,25.46%,sklearn,False,False,SVC,nan,4080.000000,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,2
1,24.00%,18.54%,22.19%,19.57%,sklearn,False,False,SVC,nan,4080.000000,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,4
2,24.00%,18.33%,19.64%,23.97%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,1
3,18.00%,13.63%,13.48%,15.83%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,3
4,24.00%,11.16%,9.65%,19.44%,keras,False,True,Sequential,4.000000,1904.000000,4.000000,3.000000,1.000000,str,SGD,8664.000000,ML_Risk_Mgmnt,5


## 5. Repeat submission process to improve place on leaderboard


In [ ]:
# Train and submit model 2 using same preprocessor (note that you could save a new preprocessor, but we will use the same one for this example).
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

model_2 = Sequential()
model_2.add(Dense(128, input_dim=34, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dense(64, activation='relu'))
model_2.add(Dropout(.3))
model_2.add(Dense(64, activation='relu'))

model_2.add(Dense(16, activation='softmax')) 
                                            
# Compile model
model_2.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

# Fitting the NN to the Training set
model_2.fit(preprocessor(X_train), y_train, 
               epochs = 50, validation_split=0.25) 

In [26]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(model_2, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("model2.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [27]:
#Submit Model 2: 

#-- Generate predicted y values (Model 2)
#Note: Keras predict returns the predicted column index location for classification models
prediction_column_index=model_2.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit Model 2 to Competition Leaderboard
mycompetition.submit_model(model_filepath = "model2.onnx",
                                 prediction_submission=prediction_labels,
                                 preprocessor_filepath="preprocessor.zip")

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 6

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1756


In [28]:
# Compare two or more models
data=mycompetition.compare_models([5,6], verbose=1)
mycompetition.stylize_compare(data)

,Model_5_Layer,Model_5_Shape,Model_5_Params,Model_6_Layer,Model_6_Shape,Model_6_Params
0,Dense,"[None, 16]",560.000000,Dense,"[None, 128]",4480
1,Dense,"[None, 16]",272.000000,Dropout,"[None, 128]",0
2,Dense,"[None, 32]",544.000000,Dense,"[None, 64]",8256
3,Dense,"[None, 16]",528.000000,Dense,"[None, 64]",4160
4,None,None,nan,Dropout,"[None, 64]",0
5,None,None,nan,Dense,"[None, 64]",4160
6,None,None,nan,Dense,"[None, 16]",1040


## Optional: Tune model within range of hyperparameters with Keras Tuner

*Simple example shown below. Consult [documentation](https://keras.io/guides/keras_tuner/getting_started/) to see full functionality.*

In [ ]:
! pip install keras_tuner

In [30]:
#Separate validation data 
from sklearn.model_selection import train_test_split
x_train_split, x_val, y_train_split, y_val = train_test_split(
     X_train, y_train, test_size=0.2, random_state=42)

In [32]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.regularizers import l1, l2, l1_l2
import keras_tuner as kt


#Define model structure & parameter search space with function
def build_model(hp):
    model = keras.Sequential()
    model.add(Dense(64, input_dim=34, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(units=hp.Int("units", min_value=32, max_value=512, step=32), #range 32-512 inclusive, minimum step between tested values is 32
                    activation='relu', kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dense(16, activation='softmax')) 
    model.compile(
        optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"],
    )
    return model

#initialize the tuner (which will search through parameters)
tuner = kt.RandomSearch(
    hypermodel=build_model, 
    objective="val_accuracy", # objective to optimize
    max_trials=3, #max number of trials to run during search
    executions_per_trial=3, #higher number reduces variance of results; guages model performance more accurately 
    overwrite=True,
    directory="tuning_model",
    project_name="tuning_units",
)

tuner.search(preprocessor(x_train_split), y_train_split, epochs=1, validation_data=(preprocessor(x_val), y_val))

Trial 3 Complete [00h 00m 05s]
val_accuracy: 0.06910569096604983

Best val_accuracy So Far: 0.10975609719753265
Total elapsed time: 00h 00m 15s


In [ ]:
# Build model with best hyperparameters

# Get the top 2 hyperparameters.
best_hps = tuner.get_best_hyperparameters(5)
# Build the model with the best hp.
tuned_model = build_model(best_hps[0])
# Fit with the entire dataset.
tuned_model.fit(x=preprocessor(X_train), y=y_train, epochs=50)

In [35]:
# Save keras model to local ONNX file
from aimodelshare.aimsonnx import model_to_onnx

onnx_model = model_to_onnx(tuned_model, framework='keras',
                          transfer_learning=False,
                          deep_learning=True)

with open("tuned_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

In [36]:
#Submit Model 3: 

#-- Generate predicted y values (Model 3)
prediction_column_index=tuned_model.predict(preprocessor(X_test)).argmax(axis=1)

# extract correct prediction labels 
prediction_labels = [y_train.columns[i] for i in prediction_column_index]

# Submit to Competition Leaderboard
mycompetition.submit_model(model_filepath = "tuned_model.onnx",
                                 preprocessor_filepath="preprocessor.zip",
                                 prediction_submission=prediction_labels)

Insert search tags to help users find your model (optional): 
Provide any useful notes about your model (optional): 

Your model has been submitted as model version 7

To submit code used to create this model or to view current leaderboard navigate to Model Playground: 

 https://www.modelshare.org/detail/model:1756


In [37]:
# Get leaderboard

data = mycompetition.get_leaderboard()
mycompetition.stylize_leaderboard(data)

,accuracy,f1_score,precision,recall,ml_framework,transfer_learning,deep_learning,model_type,depth,num_params,dropout_layers,dense_layers,relu_act,softmax_act,loss,optimizer,memory_size,username,version
0,32.00%,23.20%,28.63%,25.46%,sklearn,False,False,SVC,nan,4080.000000,nan,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,2
1,24.00%,18.54%,22.19%,19.57%,sklearn,False,False,SVC,nan,4080.000000,nan,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,4
2,24.00%,18.33%,19.64%,23.97%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,1
3,24.00%,11.16%,9.65%,19.44%,keras,False,True,Sequential,4.000000,1904.000000,nan,4.000000,3.000000,1.000000,str,SGD,8664.000000,ML_Risk_Mgmnt,5
4,18.00%,13.63%,13.48%,15.83%,sklearn,False,False,RandomForestClassifier,nan,nan,nan,nan,nan,nan,nan,nan,nan,ML_Risk_Mgmnt,3
5,22.00%,5.71%,4.59%,11.81%,keras,False,True,Sequential,7.000000,22096.000000,2.000000,5.000000,4.000000,1.000000,str,SGD,89720.000000,ML_Risk_Mgmnt,6
6,24.00%,3.23%,2.00%,8.33%,keras,False,True,Sequential,4.000000,21968.000000,nan,4.000000,3.000000,1.000000,str,RMSprop,88920.000000,ML_Risk_Mgmnt,7


In [38]:
# Compare two or more models
data=mycompetition.compare_models([5, 6, 7], verbose=1)
mycompetition.stylize_compare(data)

,Model_5_Layer,Model_5_Shape,Model_5_Params,Model_6_Layer,Model_6_Shape,Model_6_Params,Model_7_Layer,Model_7_Shape,Model_7_Params
0,Dense,"[None, 16]",560.000000,Dense,"[None, 128]",4480,Dense,"[None, 64]",2240.000000
1,Dense,"[None, 16]",272.000000,Dropout,"[None, 128]",0,Dense,"[None, 64]",4160.000000
2,Dense,"[None, 32]",544.000000,Dense,"[None, 64]",8256,Dense,"[None, 192]",12480.000000
3,Dense,"[None, 16]",528.000000,Dense,"[None, 64]",4160,Dense,"[None, 16]",3088.000000
4,None,None,nan,Dropout,"[None, 64]",0,None,None,nan
5,None,None,nan,Dense,"[None, 64]",4160,None,None,nan
6,None,None,nan,Dense,"[None, 16]",1040,None,None,nan
